In [1]:
# import packages 
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from scipy.interpolate import interp1d
%matplotlib inline

# define your own folder paths and names
project_path = os.getcwd()
input_folder = os.path.join(project_path,'data','pv_data')
output_folder = os.path.join(project_path,'data','pv_data')

# file path
pv_data_raw_path = os.path.join(input_folder,'pv_output.csv')

# Operating parameter
start_date = datetime.datetime(2017,1,1)
end_date = datetime.datetime(2019,12,31)

## 1. Read raw data from excel file

In [2]:
# load raw data
pv_data_raw_df = pd.read_csv(pv_data_raw_path)

# modify the time format to datetime
pv_data_df = pd.DataFrame()
pv_data_df["Time"] = pv_data_raw_df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S"))
pv_data_df["PV_output_kw"] = pv_data_raw_df["Huang_E4102_kW"]

# select sample from starting time
pv_data_df = pv_data_df[(pv_data_df['Time']>=start_date) & (pv_data_df['Time']<=end_date)]

# reset index to starting from 0 and drop the old index
pv_data_df.reset_index(drop=True, inplace=True)
pv_data_df = pv_data_df.set_index('Time')

In [3]:
pv_data_df

,PV_output_kw
Time,
2017-01-01 00:00:00,-0.084945
2017-01-01 00:01:00,-0.084945
2017-01-01 00:02:00,-0.084945
2017-01-01 00:03:00,-0.084945
2017-01-01 00:04:00,-0.084945
...,...
2019-10-27 08:56:00,5.983374
2019-10-27 08:57:00,6.125702
2019-10-27 08:58:00,6.291990


## 2. PV data interpolation

In [4]:
def interpaverage(record_times,record_outputs):
    # Generate interpolation function with interp1d
    start_time_dt = record_times[0]
    start_time_str = datetime.datetime.strftime(start_time_dt,"%Y-%m-%dT%H:%M:%S")
    record_times = record_times.astype('datetime64[s]')
    record_times_elapsed = (record_times - np.datetime64(start_time_str)).astype('int')
    f = interp1d(record_times_elapsed, record_outputs)

    # Interpolating to every 1 seconds 
    int_times_elapsed = np.arange(0, record_times_elapsed[-1], 1)
    int_outputs = f(int_times_elapsed)
    int_datetimes = start_time_dt+int_times_elapsed*datetime.timedelta(seconds=1)
    
    # Store the interpolated RAW data into a pd series
    pv_output_raw_int = pd.Series(int_outputs, index = int_datetimes) 

    # Store to hard drive
    pv_output_10s_int = pv_output_raw_int.resample('10S').first()

    # Assert all index is 10s*N
    assert np.sum((pv_output_10s_int.index.second % 10).values) == 0
    
    return pv_output_10s_int

In [5]:
record_times = np.asarray([time.to_pydatetime() for time in pv_data_df.index])
record_outputs = pv_data_df['PV_output_kw'].values

print("record_times.shape: ", record_times.shape)
print("record_outputs.shape: ", record_outputs.shape)

record_times.shape:  (1482241,)
record_outputs.shape:  (1482241,)


In [6]:
record_times

array([datetime.datetime(2017, 1, 1, 0, 0),
       datetime.datetime(2017, 1, 1, 0, 1),
       datetime.datetime(2017, 1, 1, 0, 2), ...,
       datetime.datetime(2019, 10, 27, 8, 58),
       datetime.datetime(2019, 10, 27, 8, 59),
       datetime.datetime(2019, 10, 27, 9, 0)], dtype=object)

In [7]:
record_outputs

array([-0.08494503, -0.08494503, -0.08494503, ...,  6.2919903 ,
        6.38645795,  6.52732201])

In [8]:
pv_output_10s_int = interpaverage(record_times,record_outputs)
pv_output_10s_int

C:\Users\aiden\AppData\Local\Temp\ipykernel_28304\1874213284.py:18: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  pv_output_10s_int = pv_output_raw_int.resample('10S').first()


2017-01-01 00:00:00   -0.084945
2017-01-01 00:00:10   -0.084945
2017-01-01 00:00:20   -0.084945
2017-01-01 00:00:30   -0.084945
2017-01-01 00:00:40   -0.084945
                         ...   
2019-10-27 08:59:10    6.409935
2019-10-27 08:59:20    6.433413
2019-10-27 08:59:30    6.456890
2019-10-27 08:59:40    6.480367
2019-10-27 08:59:50    6.503845
Freq: 10s, Length: 8893800, dtype: float64

## 3. Plot PV profile for everyday: Visual inspection of invalid days 

In [ ]:
#  Step through the PV_data day by day 
xfmt = mdates.DateFormatter('%H')
for date in pd.date_range(start = pv_output_10s_int.index[0], end = pv_output_10s_int.index[-1],
                         freq = '1D', inclusive = 'left'):
    hours_xaxis = pv_output_10s_int.loc[date:date+datetime.timedelta(days = 1)].index
    pv_one_day = pv_output_10s_int.loc[date:date+datetime.timedelta(days = 1)].values
    f, ax = plt.subplots()
    print('plotting for date',date.strftime('%Y%m%d'))
    ax.plot(hours_xaxis,pv_one_day,'-o',markerfacecolor='None')
    ax.grid(True)
    ax.set_xlabel('Time of the day ')
    ax.set_ylabel('panel output (kW)')
    ax.xaxis.set_major_formatter(xfmt)
    plt.show()

TypeError: DatetimeArray._generate_range() got an unexpected keyword argument 'closed'

In [11]:
# define invalide dates based on the curves above for sanity check
# end date is not included
# for example, the invalid days are [2017-03-10,2017-03-13), essentially [2017-03-10, 2017-03-12]
invalid_dates = {'start':[datetime.datetime(2017,3,10), datetime.datetime(2017,3,13), datetime.datetime(2017,3,18), 
datetime.datetime(2017,3,31), datetime.datetime(2017,4,5,8,34,0), datetime.datetime(2017,5,6,18,50,0), datetime.datetime(2017,7,17), 
datetime.datetime(2017,9,8,8,59,0), datetime.datetime(2017,10,9,10,3,0), datetime.datetime(2017,11,5), datetime.datetime(2017,12,30), 
datetime.datetime(2018,1,1), datetime.datetime(2018,3,11), datetime.datetime(2018,4,2),datetime.datetime(2018,6,13), 
datetime.datetime(2018,8,22), datetime.datetime(2018,9,25), datetime.datetime(2018,10,16), datetime.datetime(2018,11,4), 
datetime.datetime(2019, 1, 28, 10, 29, 0), datetime.datetime(2019, 5, 22, 14, 18, 0), datetime.datetime(2019, 10, 27)],
'end':[datetime.datetime(2017,3,13), datetime.datetime(2017,3,16), datetime.datetime(2017,3,20), datetime.datetime(2017,4,1), 
datetime.datetime(2017,4,5,9,51,0), datetime.datetime(2017,5,6,21,8,0), datetime.datetime(2017,7,18), datetime.datetime(2017,9,8,11,26,0), 
datetime.datetime(2017,10,9,12,12,0), datetime.datetime(2017,11,6), datetime.datetime(2018,1,1), datetime.datetime(2018,1,5), 
datetime.datetime(2018,3,13), datetime.datetime(2018,4,14),datetime.datetime(2018,6,14), datetime.datetime(2018,9,18), 
datetime.datetime(2018,10,1,6,0,0), datetime.datetime(2018,10,17), datetime.datetime(2018, 11, 26), datetime.datetime(2019, 1, 28, 11, 47, 0), 
datetime.datetime(2019, 5, 22, 17, 3, 0), datetime.datetime(2019, 10, 28)]}

## 4. Filter for invalid data

### 4.1 Invalid dates filter

In [12]:
invalid_dates_mask = pd.Series(False, index = pv_output_10s_int.index)

for i in range(len(invalid_dates["start"])):
    start_idx = pd.Timestamp(invalid_dates['start'][i])
    end_idx = pd.Timestamp(invalid_dates['end'][i])    
    invalid_dates_mask.loc[start_idx:end_idx] = True

### 4.2 Missing record filter

In [13]:
# Calculate raw record interval
record_interval = record_times[1:] - record_times[:-1]

# Consider interval > 1 hour as invalid (i.e., linear interpolation for > 1 hour is invalid)
record_invalid_start = record_times[:-1][record_interval > np.timedelta64(1,'h')]
record_invalid_end = record_times[1:][record_interval > np.timedelta64(1,'h')]

missing_mask = pd.Series(False, index = pv_output_10s_int.index)

for start,end in zip(record_invalid_start, record_invalid_end):
    missing_mask.loc[start:end] = True

In [14]:
# check the invalid time stamps
for start, end in zip(record_invalid_start,record_invalid_end):
    print(start, end)

2017-03-12 01:59:00 2017-03-12 03:00:00
2018-03-11 01:59:00 2018-03-11 03:00:00
2019-03-10 01:59:00 2019-03-10 03:00:00


### 4.3 Night filter (PV_output<0)

In [15]:
pv_output_valid = pv_output_10s_int[(~invalid_dates_mask) & (~missing_mask) & (pv_output_10s_int>0)]
pv_output_valid

2017-01-03 08:14:20    0.010697
2017-01-03 08:14:30    0.023494
2017-01-03 08:14:40    0.036292
2017-01-03 08:14:50    0.049089
2017-01-03 08:15:00    0.061887
                         ...   
2019-10-26 17:58:40    0.020961
2019-10-26 17:58:50    0.015540
2019-10-26 17:59:00    0.010119
2019-10-26 17:59:10    0.005942
2019-10-26 17:59:20    0.001765
Length: 3884410, dtype: float64

In [ ]:
#  Inspect again for the valid data
xfmt = mdates.DateFormatter('%H')
all_times = pv_output_valid.index
all_pv = pv_output_valid.values
all_dates = np.asarray([times.date() for times in all_times])
unique_dates = np.unique(all_dates)

for date in unique_dates:
    idx = np.where(all_dates==date)[0]
    time_one_day = all_times[idx]
    pv_one_day = all_pv[idx]
    f, ax = plt.subplots()
    print('plotting for date',date.strftime('%Y-%m-%d'))
    ax.plot(time_one_day,pv_one_day,'-o',markerfacecolor='None')
    ax.grid(True)
    ax.set_xlabel('Time of the day')
    ax.set_ylabel('panel output (kW)')
    ax.xaxis.set_major_formatter(xfmt)
    plt.show()

In [17]:
# Store the PV output record. Freq = 10s
pv_output_valid.to_pickle(os.path.join(output_folder,'pv_output_valid.pkl'))